In [1]:
import os
import sys
import time
import re
from typing import List, Union, Dict, Any

sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

!pip install transformers[torch] datasets
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install --upgrade langchain
!pip install langchain-community
!pip install google-search-results


import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, GenerationConfig, pipeline)
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification

from langchain import (HuggingFacePipeline, LLMChain)
from langchain.agents import (load_tools, initialize_agent, Tool,
                               AgentExecutor, LLMSingleActionAgent,
                               AgentOutputParser)
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from datasets import load_dataset

sys.stdout = sys.__stdout__
sys.stderr = sys.__stderr__

os.environ["SERPER_API_KEY"] = "5d73a1cb275e08283e0c3b7d6743c4390dc26e5a"

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Model
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

def load_model(model_name):
  quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                          bnb_4bit_compute_dtype=torch.bfloat16,
                                          bnb_4bit_use_double_quant=True,
                                          bnb_4bit_quant_type= "nf4"
                                          )

  quantized_model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map="auto",
                    torch_dtype=torch.bfloat16,
                    quantization_config=quantization_config
                    )

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  return quantized_model, tokenizer

model, tokenizer = load_model(MODEL_NAME)
model.eval()

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [5]:
dataset = load_dataset("cais/mmlu", "all")
data = dataset.filter(lambda example: example["subject"] == "college_mathematics")

In [6]:
def generate_response(prompt, max_new_tokens, temperature):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            inputs["input_ids"],
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response.strip()


def calculate_accuracy(actual_answers, predicted_answers):
  cc = 0
  for i in range(len(actual_answers)):
    if actual_answers[i] == predicted_answers[i]:
      cc += 1
  return cc/len(actual_answers)


def extract_correct_option_with_same_llm(text):
    prompt = text
    prompt += 'The correct option is Option:'
    return generate_response(prompt, 1, 0.2)


classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def extract_correct_option_with_bart(text):
    candidate_labels = ["Answer Option A is correct and final", "Answer Option B is correct and final", "Answer Option C is correct and final", "Answer Option D is correct and final"]
    result = classifier(text, candidate_labels)
    predicted_option = result['labels'][0]
    return predicted_option.split()[2]


deberta = "microsoft/deberta-v3-large"
deberta_tokenizer = DebertaV2Tokenizer.from_pretrained(deberta)
deberta_model = DebertaV2ForSequenceClassification.from_pretrained(deberta)

def extract_correct_option_with_debarta(text, options=['A', 'B', 'C', 'D']):
    hypothesis_template = "The correct and final answer is option {}."
    candidate_labels = [hypothesis_template.format(option) for option in options]
    inputs = deberta_tokenizer(text, candidate_labels, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = deberta_model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return options[predicted_class]


In [13]:
def zeroshot_inference(dataset):
    ground_truth_answers = []
    model_predicted_answers = []
    model_responses = []
    extracted_answers = []
    processed_answers = []

    start_time = time.time()

    for example in dataset:
        question = example["question"]
        choices = example["choices"]
        choice_a, choice_b, choice_c, choice_d = choices
        correct_answer = example["answer"]

        prompt = f"MCQ Question: {question}\n"
        prompt += "Four available options:\n"
        prompt += f"Option A. {choice_a}\nOption B. {choice_b}\nOption C. {choice_c}\nOption D. {choice_d}\n"
        prompt += "Return correct option in format 'Option X'."

        raw_response = generate_response(prompt, max_new_tokens=512, temperature=0.2)
        response = raw_response.split("Return correct option in format 'Option X'")[1].strip()

        try:
            extracted_answer = extract_correct_option_with_bart(response)
        except Exception as e:
            extracted_answer = 'N/A'

        extracted_answers.append(extracted_answer)

        option_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

        if extracted_answer in option_to_index:
            predicted_answer = option_to_index[extracted_answer]
        else:
            predicted_answer = extracted_answer

        ground_truth_answers.append(correct_answer)
        model_predicted_answers.append(predicted_answer)
        model_responses.append(response)

    end_time = time.time()
    total_duration = end_time - start_time

    accuracy = calculate_accuracy(ground_truth_answers, model_predicted_answers)

    return accuracy, ground_truth_answers, model_predicted_answers, model_responses, extracted_answers, processed_answers, total_duration

zero_inference_accuracy, zero_actual_answers, zero_predicted_answers, zero_responses, zero_extracted_answers, zero_processed_answers, zero_inference_duration = zeroshot_inference(data['test'].select(range(10)))
zero_inference_results = {
    'zero_accuracy': zero_inference_accuracy,
    'zero_duration': zero_inference_duration,
    'zero_actual_answers': zero_actual_answers,
    'zero_predicted_answers': zero_predicted_answers,
}
zero_inference_results

{'zero_accuracy': 0.4,
 'zero_duration': 584.9557974338531,
 'zero_actual_answers': [1, 3, 3, 0, 2, 3, 2, 2, 2, 0],
 'zero_predicted_answers': [1, 1, 0, 0, 0, 3, 0, 2, 3, 2]}

In [14]:
zero_responses

[". (e.g. 'Option A').\n\nSolution: The equation e^x + x - 2 = 0 has a unique solution in the interval [0, 1] (which is x = 0.32), and no other real solutions. Therefore, k = 1 and n = 0. The correct answer is 'Option B'. (Note: The equation e^x + x - 2 = 0 has no real solutions for x > 1, and has two real solutions for x < 0, but these are not in the interval [0, 1].) Final Answer: The final answer is Option B. I hope it is correct.",
 ". For example, 'Option B'. If you want to provide a brief explanation, you can do so in the comments section.\n\nFinal Answer: The final answer is Option B. I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it is correct.  I hope it 

In [15]:
def chainshot_inference(dataset):
    ground_truth_answers = []
    model_predicted_answers = []
    model_responses = []
    extracted_answers = []
    processed_answers = []

    start_time = time.time()

    for example in dataset:
        question = example["question"]
        choices = example["choices"]
        choice_a, choice_b, choice_c, choice_d = choices
        correct_answer = example["answer"]

        prompt = f"MCQ Question: {question}\n"
        prompt += "Four available options:\n"
        prompt += f"Option A. {choice_a}\nOption B. {choice_b}\nOption C. {choice_c}\nOption D. {choice_d}\n"
        prompt += "Think step by step, it is a math question."
        prompt += "Return correct option in format 'Option X'."

        raw_response = generate_response(prompt, max_new_tokens=512, temperature=0.2)
        response = raw_response.split("Return correct option in format 'Option X'")[1].strip()

        try:
            extracted_answer = extract_correct_option_with_bart(response)
        except Exception as e:
            extracted_answer = 'N/A'

        extracted_answers.append(extracted_answer)

        option_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

        if extracted_answer in option_to_index:
            predicted_answer = option_to_index[extracted_answer]
        else:
            predicted_answer = extracted_answer

        ground_truth_answers.append(correct_answer)
        model_predicted_answers.append(predicted_answer)
        model_responses.append(response)

    end_time = time.time()
    total_duration = end_time - start_time

    accuracy = calculate_accuracy(ground_truth_answers, model_predicted_answers)

    return accuracy, ground_truth_answers, model_predicted_answers, model_responses, extracted_answers, processed_answers, total_duration

chain_accuracy, chain_actual_answers, chain_predicted_answers, chain_responses, chain_extracted_answers, chain_processed_answers, chain_duration = chainshot_inference(data['test'].select(range(10)))
chain_results = {
    'chain_accuracy': chain_accuracy,
    'chain_duration': chain_duration,
    'chain_actual_answers': chain_actual_answers,
    'chain_predicted_answers': chain_predicted_answers,
}
chain_results

{'chain_accuracy': 0.4,
 'chain_duration': 779.782178401947,
 'chain_actual_answers': [1, 3, 3, 0, 2, 3, 2, 2, 2, 0],
 'chain_predicted_answers': [0, 1, 0, 0, 3, 0, 2, 2, 3, 0]}

In [16]:
chain_responses

[". For example, if you think the correct option is 'Option A', you should write 'Option A'. If you think the correct option is 'Option B', you should write 'Option B'. If you think the correct option is 'Option C', you should write 'Option C'. If you think the correct option is 'Option D', you should write 'Option D'. If you think the correct option is not among the given options, you should write 'None of the above'. If you think the correct option is not among the given options, you should write 'None of the above'. If you think the correct option is not among the given options, you should write 'None of the above'. If you think the correct option is not among the given options, you should write 'None of the above'. If you think the correct option is not among the given options, you should write 'None of the above'. If you think the correct option is not among the given options, you should write 'None of the above'. If you think the correct option is not among the given options, you

In [17]:
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]
    input_variables: List[str] = ["input", "intermediate_steps"]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps", [])
        thoughts = "".join(
            f"{action.log}\nObservation: {observation}\nThought: "
            for action, observation in intermediate_steps
        )
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Option corresponding to correct answer is:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output},
                log=llm_output,
            )
        match = re.search(r"Action: (.*?)\nAction Input:[\s]*(.*)", llm_output, re.DOTALL)
        if not match:
            return AgentFinish(return_values={"output": llm_output}, log=llm_output)
        return AgentAction(
            tool=match.group(1).strip(),
            tool_input=match.group(2).strip(),
            log=llm_output,
        )

tools = [
    Tool.from_function(
        name="llm-math",
        description="Mathematical operations using an LLM.",
        func=llm
    ),
    Tool.from_function(
        name="google-serper",
        description="Search using Google Serper.",
        func=lambda x: x
    )
]

prompt = CustomPromptTemplate(
    template="""Answer the following question using these tools:

{tools}

Use this format:
Question: {input}
Thought: Think what to do next
Action: Pick an action [{tool_names}]
Action Input: Provide the input
Observation: Result of action
... (repeat as needed)
Thought: I know the final answer
Option corresponding to correct answer is: The option

Question: {input}
{agent_scratchpad}""",
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=CustomOutputParser(),
    stop=["\nObservation:"],
    allowed_tools=[tool.name for tool in tools]
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=1,
)

In [20]:
def react_inference(dataset):
    ground_truth_answers = []
    model_predicted_answers = []
    extracted_answers = []
    agent_outputs = []
    processed_answers = []

    start_time = time.time()

    for example in dataset:
        question = example["question"]
        choices = example["choices"]
        choice_a, choice_b, choice_c, choice_d = choices
        correct_answer = example["answer"]

        prompt = f"MCQ Question: {question}\n"
        prompt += "Four available options:\n"
        prompt += f"Option A. {choice_a}\nOption B. {choice_b}\nOption C. {choice_c}\nOption D. {choice_d}\n"
        prompt += "Return correct option in format 'Option X'."

        agent_output = agent_executor.run(prompt)
        agent_outputs.append(agent_output)

        try:
            text = agent_output.strip().split('\n\nQuestion: ')[1].strip().split("Return correct option in format 'Option X'.")[1].strip()
        except Exception as e:
            text = None

        if text:
            try:
              extracted_answer = extract_correct_option_with_bart(text)
            except Exception as e:
              extracted_answer = 'N/A'
        else:
            extracted_answer = 'N/A'

        extracted_answers.append(extracted_answer)

        option_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
        if extracted_answer in option_to_index:
            predicted_answer = option_to_index[extracted_answer]
        else:
            predicted_answer = extracted_answer

        ground_truth_answers.append(correct_answer)
        model_predicted_answers.append(predicted_answer)

    end_time = time.time()
    total_duration = end_time - start_time

    accuracy = calculate_accuracy(ground_truth_answers, model_predicted_answers)

    return accuracy, ground_truth_answers, model_predicted_answers, agent_outputs, extracted_answers, processed_answers, total_duration


react_accuracy, react_actual_answers, react_predicted_answers, react_agent_outputs, react_extracted_answers, react_processed_answers, react_duration = react_inference(data['test'].select(range(10)))
react_results = {
    'react_accuracy': react_accuracy,
    'react_duration': react_duration,
    'react_actual_answers': react_actual_answers,
    'react_predicted_answers': react_predicted_answers,
}
react_results

{'react_accuracy': 0.3,
 'react_duration': 732.1060891151428,
 'react_actual_answers': [1, 3, 3, 0, 2, 3, 2, 2, 2, 0],
 'react_predicted_answers': [1, 1, 0, 1, 2, 1, 0, 2, 3, 2]}

In [21]:
react_agent_outputs

["Answer the following question using these tools:\n\nllm-math: Mathematical operations using an LLM.\ngoogle-serper: Search using Google Serper.\n\nUse this format:\nQuestion: MCQ Question: Let k be the number of real solutions of the equation e^x + x - 2 = 0 in the interval [0, 1], and let n be the number of real solutions that are not in [0, 1]. Which of the following is true?\nFour available options:\nOption A. k = 0 and n = 1\nOption B. k = 1 and n = 0\nOption C. k = n = 1\nOption D. k > 1\nReturn correct option in format 'Option X'.\nThought: Think what to do next\nAction: Pick an action [llm-math, google-serper]\nAction Input: Provide the input\nObservation: Result of action\n... (repeat as needed)\nThought: I know the final answer\nOption corresponding to correct answer is: The option\n\nQuestion: MCQ Question: Let k be the number of real solutions of the equation e^x + x - 2 = 0 in the interval [0, 1], and let n be the number of real solutions that are not in [0, 1]. Which of 